In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',None)

data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

# 데이터 전처리 파이프라인

In [12]:
import itertools
from tqdm import tqdm
from datetime import datetime, timedelta
pd.set_option('mode.chained_assignment',  None)
class Processing:
    def __init__(self, x, test): # 이후 테스트 데이터도 넣는 버전 만들어야함
        self.x = x
        self.test = test
    
    def fill_airline_and_id_2(self, data):
        master_dil = 'MASTER2.txt'
        df = []
        with open(master_dil, 'r', encoding = "utf-8-sig") as file:
           df.append(file.readlines())
           
        df = list(itertools.chain(*df))  
        df = [line.split(',') for line in df]
        col = df[0]
        df = pd.DataFrame(df, columns = col)
        Airline2 = df[['N-NUMBER', 'NAME']].rename(columns={'N-NUMBER' : 'Tail_Number', 'NAME' : 'NAME2'})
        df['Tail_Number'] = ['N' + i for i in df['N-NUMBER']]
        Airline = df[['Tail_Number', 'NAME']]

        data = pd.merge(data, Airline, left_on = 'Tail_Number', right_on = 'Tail_Number', how = 'left')
        ar = pd.merge(data, Airline2, left_on = 'Tail_Number', right_on = 'Tail_Number', how = 'left')
        
        ar.loc[ar['Tail_Number'] == 'N297AK', 'NAME'] = 'ALASKA AIRLINES INC'
        ar.loc[ar['Tail_Number'] == '276NV', 'NAME'] = 'ALLEGIANT AIR LLC'
        
        ar = ar.fillna('nan')
        
        # DB-NAME을 Train Airline 기준으로 바꿈
        for base in ar.loc[~ar['Airline'].isin(['nan']), 'Airline'].unique():
            name1 = ar.loc[(ar['Airline'] == base) & ~(ar['NAME'].isin(['nan'])) & ~(ar['NAME'].isin(ar['Airline'].unique())), 'NAME'].unique()
            name2 = ar.loc[(ar['Airline'] == base) & ~(ar['NAME2'].isin(['nan'])) & ~(ar['NAME2'].isin(ar['Airline'].unique())), 'NAME2'].unique()
            ar['NAME'].replace(name1, base, inplace=True)
            ar['NAME2'].replace(name2, base, inplace=True)
        
        def apy(x):
            if x['NAME'] != 'nan':
                x['Airline'] = x['NAME']
            elif (x['NAME'] == 'nan') & (x['NAME2'] != 'nan'):
                x['Airline'] = x['NAME2']
            return x
        
        ar.loc[ar['Airline'] == 'nan'] = ar.loc[ar['Airline'] == 'nan'].apply(apy, axis=1)
        
        for n in ar[ar['Airline'] == 'nan']['Tail_Number'].unique():
            if ar.loc[ar['Tail_Number'] == n, ['Airline', 'Carrier_ID(DOT)']].shape[0] != ar.loc[(ar['Tail_Number'] == n) & ~(ar['Airline'].isin(['nan'])), ['Airline', 'Carrier_ID(DOT)']].shape[0]:
                ar.loc[ar['Tail_Number'] == n, 'Airline'] = ar.loc[(ar['Tail_Number'] == n) & ~(ar['Airline'].isin(['nan'])), 'Airline'].unique()[0]
                ar.loc[ar['Tail_Number'] == n, 'Carrier_ID(DOT)'] = ar.loc[(ar['Tail_Number'] == n) & ~(ar['Airline'].isin(['nan'])), 'Carrier_ID(DOT)'].unique()[0]
        
        ar['Airline'] = ar['Airline'].str.rstrip()
        ar['Airline'] = ar['Airline'].str.split('.').str[0]
        
        # ID 다시채움
        for airline in ar['Airline'].unique():
            id = ar.loc[(ar['Airline'] == airline) & ~(ar['Carrier_ID(DOT)'].isin(['nan'])), 'Carrier_ID(DOT)'].unique()
            ar.loc[ar['Airline'] == airline, 'Carrier_ID(DOT)'] = id[0]
        
        return ar
                
    def to_time(self, data):
        def to_time(time_list):
            Time = pd.Series(time_list).astype(str).str.zfill(4)
            Time = Time.replace('2400','0000')
            return [datetime.strptime(i, '%H%M').strftime("%H:%M") if i != '0nan' else np.NaN for i in Time] 
        
        def time_gb(x):
            if x >= 600 and x <= 659:
                return '0600-0659'
            elif x>=1400 and x<=1459:
                return '1400-1459'
            elif x>=1200 and x<=1259:
                return '1200-1259'
            elif x>=1500 and x<=1559:
                return '1500-1559'
            elif x>=1900 and x<=1959:
                return '1900-1959'
            elif x>=900 and x<=959:
                return '0900-0959'
            elif x>=1000 and x<=1059:
                return  '1000-1059'
            elif x>=2000 and x<=2059:
                return '2000-2059'
            elif x>=1300 and x<=1359:
                return '1300-1359'
            elif x>=1100 and x<=1159:
                return '1100-1159'
            elif x>=800 and x<=859:
                return '0800-0859'
            elif x>=2200 and x<=2259:
                return '2200-2259'
            elif x>=1600 and x<=1659:
                return '1600-1659'
            elif x>=1700 and x<=1759:
                return '1700-1759'
            elif x>=2100 and x<=2159:
                return '2100-2159'
            elif x>=700 and x<=759:
                return '0700-0759'
            elif x>=1800 and x<=1859:
                return '1800-1859'
            elif x>=1 and x<=559:
                return '0001-0559'
            elif x>=2300 and x<=2400:
                return '2300-2400'
            
        def distance_gb(x):
            if x < 700 :
                return 'short_route'
            elif x>=700 and x<3000:
                return 'mid_route'
            elif x>=3000:
                return 'long_route'    
            
        time_list = [i if str(i) == 'nan' else str(int(i))  for i in data['Estimated_Departure_Time'] ]
        time_list1 = [i if str(i) == 'nan' else str(int(i))  for i in data['Estimated_Arrival_Time'] ]
        data['Estimated_Departure_Time_HH:MM'] = to_time(time_list)
        data['Estimated_Arrival_Time_HH:MM'] = to_time(time_list1)
    
        
        # 예상 비행시간 만들기 (분으로 만들기)
        data_est_time = []
        for i,j in zip(data['Estimated_Arrival_Time_HH:MM'], data['Estimated_Departure_Time_HH:MM']):
            if str(i) != 'nan' and str(j) != 'nan':
                if (datetime.strptime(str(i), "%H:%M") - datetime.strptime(str(j), "%H:%M")).total_seconds()/60 >= 0:
                    data_est_time.append((datetime.strptime(str(i), "%H:%M") - datetime.strptime(str(j), "%H:%M")).total_seconds()/60)
                else:
                    time = datetime.strptime(str(i), "%H:%M") - datetime.strptime(str(j), "%H:%M") + datetime.strptime('23:59', "%H:%M") + timedelta(minutes=1)
                    data_est_time.append(timedelta(hours=time.hour,minutes=time.minute ).total_seconds()/60)
            else:
                data_est_time.append(np.NaN)
                
        data['Estimated_Time'] = data_est_time
        data['Dep_time_gb'] = data['Estimated_Departure_Time'].dropna().apply(time_gb)
        data['Arr_time_gb'] = data['Estimated_Arrival_Time'].dropna().apply(time_gb)
        data['route_gb'] = data['Distance'].apply(distance_gb)
        
        return data
        
    
    def dummy(self):
        data = self.fill_airline_and_id(self.x)
        data = self.fill_state(data)
        data = self.to_time(data)
        data = self.fill_airline_and_id_2(data)
        # data = self.fill_time(data)
        # data = self.to_time(data)
        return data
    
    def fill_airline_and_id(self, data):
        
        for airline in data['Airline'].dropna().unique():
            id = data.loc[data['Airline'] == airline, 'Carrier_ID(DOT)'].dropna().unique()
            data.loc[data['Airline'] == airline, 'Carrier_ID(DOT)'] = id[0]
                

        for id in data['Carrier_ID(DOT)'].dropna().unique():
            airline = data.loc[data['Carrier_ID(DOT)'] == id, 'Airline'].dropna().unique()
            data.loc[data['Carrier_ID(DOT)'] == id, 'Airline'] = airline[0]
            
        return data
    
    def fill_state(self, data):
        for id in data['Origin_Airport_ID'].dropna().unique():
            data.loc[data['Origin_Airport_ID'] == id, 'Origin_State'] = data.loc[data['Origin_Airport_ID'] == id, 'Origin_State'].dropna().unique()[0]
        
        for id in data['Destination_Airport_ID'].dropna().unique():
            try:
                data.loc[data['Destination_Airport_ID'] == id, 'Destination_State'] = data.loc[data['Destination_Airport_ID'] == id, 'Destination_State'].dropna().unique()[0]
            except: # 기록이 하나밖에 없음. Youngstown (YNG 공항)
                data.loc[data['Destination_Airport_ID'] == id, 'Destination_State'] = 'Youngstown'
        return data
        
        
    def fill_time(self, data):
        check = data[~((data['Estimated_Departure_Time'].isin(['nan'])) | (data['Estimated_Arrival_Time'].isin(['nan'])))]
        
        # 출발시간
        print('출발시간')
        iddata = data[data['Estimated_Departure_Time'] == 'nan'].index
        for i in tqdm(iddata):
            at = data.loc[i, 'Estimated_Arrival_Time']
            dt = data.loc[i, 'Estimated_Departure_Time']
            oa = data.loc[i, 'Origin_Airport']
            da = data.loc[i, 'Destination_Airport']
            al = data.loc[i, 'Airline']
            
            check_data = check[(check['Origin_Airport'] == oa) & (check['Destination_Airport'] == da) & (check['Airline'] == al)]
            # print(check_data)
            
            # 동일 시간 존재 시 동일한 수로 대체 (복수 존재 시 제일 많은걸로)
            insert_time = check_data.loc[check_data['Estimated_Arrival_Time'] == at, 'Estimated_Departure_Time']
            if len(insert_time) != 0:
                try:
                    data.loc[i, 'Estimated_Departure_Time'] = insert_time.value_counts().index.drop('nan')[0]
                
                except:
                    data.loc[i, 'Estimated_Departure_Time'] = insert_time.value_counts().index[0]
                    

            # 가장 빈도수 높은 경과시간으로 계산하여 대체
            elif len(insert_time) == 0:
                est_time = check_data['Estimated_Time'].value_counts().index[0]
                    
                hour = (datetime.strptime(at, "%H:%M") - timedelta(minutes=est_time)).hour
                minute = (datetime.strptime(at, "%H:%M") - timedelta(minutes=est_time)).minute
                data.loc[i, 'Estimated_Departure_Time'] =  f'{hour}{minute}'
                data.loc[i, 'Estimated_Departure_Time'] = data.loc[i, 'Estimated_Departure_Time'].zfill(4)
                
        # 도착시간
        print('도착시간')
        iddata = data[data['Estimated_Arrival_Time'] == 'nan'].index
        for i in tqdm(iddata):
            at = data.loc[i, 'Estimated_Arrival_Time']
            dt = data.loc[i, 'Estimated_Departure_Time']
            oa = data.loc[i, 'Origin_Airport']
            da = data.loc[i, 'Destination_Airport']
            al = data.loc[i, 'Airline']
            
            check_data = check[(check['Origin_Airport'] == oa) & (check['Destination_Airport'] == da) & (check['Airline'] == al)]
            # 동일 시간 존재 시 동일한 수로 대체 (복수 존재 시 제일 많은걸로)
            insert_time = check_data.loc[check_data['Estimated_Departure_Time'] == dt, 'Estimated_Arrival_Time']
            if len(insert_time) != 0:
                try:
                    data.loc[i, 'Estimated_Arrival_Time'] = insert_time.value_counts().index.drop('nan')[0]
                
                except:
                    data.loc[i, 'Estimated_Arrival_Time'] = insert_time.value_counts().index[0]
                    

            # 가장 빈도수 높은 경과시간으로 계산하여 대체
            elif len(insert_time) == 0:
                est_time = check_data['Estimated_Time'].value_counts().index[0]
                    
                hour = (datetime.strptime(dt, "%H:%M") + timedelta(minutes=est_time)).hour
                minute = (datetime.strptime(dt, "%H:%M") + timedelta(minutes=est_time)).minute
                data.loc[i, 'Estimated_Arrival_Time'] =  f'{hour}{minute}'
                data.loc[i, 'Estimated_Arrival_Time'] = data.loc[i, 'Estimated_Arrival_Time'].zfill(4)
            
        # 둘다 없을 떄 
        # 그냥 빈도수 많은걸로 대체
        iddata = data[((data['Estimated_Departure_Time'].isin(['nan'])) & (data['Estimated_Arrival_Time'].isin(['nan'])))].index
        for i in tqdm(iddata):
            at = data.loc[i, 'Estimated_Arrival_Time']
            dt = data.loc[i, 'Estimated_Departure_Time']
            oa = data.loc[i, 'Origin_Airport']
            da = data.loc[i, 'Destination_Airport']
            al = data.loc[i, 'Airline']
            
            check_data = check[(check['Origin_Airport'] == oa) & (check['Destination_Airport'] == da) & (check['Airline'] == al)]
            data.loc[i, 'Estimated_Departure_Time'] = check_data[['Estimated_Departure_Time',	'Estimated_Arrival_Time']].value_counts().index[0][0]
            data.loc[i, 'Estimated_Arrival_Time'] = check_data[['Estimated_Departure_Time',	'Estimated_Arrival_Time']].value_counts().index[0][1]
    
        return data
    
    
    def testset_processing(self):
        pass
    
    
                
                
            
        

In [13]:
pc = Processing(data, test)

train = pc.dummy()

In [81]:

print(len(['Southwest Airlines Co.', 'SkyWest Airlines Inc.',
       'American Airlines Inc.', 'United Air Lines Inc.',
       'Republic Airlines', 'ExpressJet Airlines Inc.',
       'Delta Air Lines Inc.', 'Air Wisconsin Airlines Corp',
       'Horizon Air', 'JetBlue Airways', 'Spirit Air Lines',
       'Alaska Airlines Inc.', 'Mesa Airlines Inc.',
       'Frontier Airlines Inc.', 'Trans States Airlines',
       'Endeavor Air Inc.', 'Comair Inc.', 'Capital Cargo International',
       'Envoy Air', 'Hawaiian Airlines Inc.', 'Allegiant Air', 'nan',
       'Virgin America', 'Peninsula Airways Inc.', 'Compass Airlines',
       'GoJet Airlines, LLC d/b/a United Express',
       'Commutair Aka Champlain Enterprises, Inc.',
       'Empire Airlines Inc.', 'Cape Air']))

print(train['Airline'].unique())
print(train['NAME'].unique())
print(train['NAME2'].unique())
print(train['Carrier_ID(DOT)'].unique())


29
['Southwest Airlines Co' 'SkyWest Airlines Inc' 'American Airlines Inc'
 'United Air Lines Inc' 'Republic Airlines' 'ExpressJet Airlines Inc'
 'Delta Air Lines Inc' 'Air Wisconsin Airlines Corp' 'Horizon Air'
 'JetBlue Airways' 'Spirit Air Lines' 'Alaska Airlines Inc'
 'Mesa Airlines Inc' 'Frontier Airlines Inc' 'Trans States Airlines'
 'Endeavor Air Inc' 'Comair Inc' 'Capital Cargo International' 'Envoy Air'
 'Hawaiian Airlines Inc' 'Allegiant Air' 'Virgin America'
 'Peninsula Airways Inc' 'Compass Airlines'
 'GoJet Airlines, LLC d/b/a United Express'
 'Commutair Aka Champlain Enterprises, Inc' 'Empire Airlines Inc'
 'Cape Air']
['Southwest Airlines Co.' 'SkyWest Airlines Inc.' 'United Air Lines Inc.'
 'Republic Airlines' 'Air Wisconsin Airlines Corp' 'Horizon Air'
 'JetBlue Airways' 'Alaska Airlines Inc.' 'nan' 'American Airlines Inc.'
 'Comair Inc.' 'Delta Air Lines Inc.' 'Hawaiian Airlines Inc.'
 'Mesa Airlines Inc.' 'Capital Cargo International'
 'Peninsula Airways Inc.' 'Spiri

In [11]:
train[((train['Estimated_Departure_Time'].isin(['nan'])) & (train['Estimated_Arrival_Time'].isin(['nan'])))]

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Time_HH:MM,Estimated_Arrival_Time_HH:MM,Estimated_Time,Dep_time_gb,Arr_time_gb,route_gb,NAME,NAME2
0,TRAIN_000000,4,15,nan,nan,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co,WN,19393.0,N7858A,nan,nan,nan,nan,nan,nan,short_route,Southwest Airlines Co.,nan
109,TRAIN_000109,3,29,nan,nan,0,0,PDX,14057,Oregon,DEN,11292,Colorado,991.0,Southwest Airlines Co,WN,19393.0,N499WN,nan,nan,nan,nan,nan,nan,mid_route,Southwest Airlines Co.,nan
277,TRAIN_000277,8,9,nan,nan,0,0,MKE,13342,Wisconsin,IAH,12266,Texas,984.0,Mesa Airlines Inc,UA,20378.0,N88335,nan,nan,nan,nan,nan,nan,mid_route,Mesa Airlines Inc.,nan
435,TRAIN_000435,9,12,nan,nan,0,0,OAK,13796,California,ORD,13930,Illinois,1836.0,Spirit Air Lines,NK,20416.0,N618NK,nan,nan,nan,nan,nan,nan,mid_route,Southwest Airlines Co.,nan
437,TRAIN_000437,7,12,nan,nan,0,0,HOU,12191,Texas,MSY,13495,Louisiana,302.0,Southwest Airlines Co,WN,19393.0,N265WN,nan,nan,nan,nan,nan,nan,short_route,Southwest Airlines Co.,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999771,TRAIN_999771,7,10,nan,nan,0,0,FLL,11697,Florida,ORD,13930,Illinois,1182.0,JetBlue Airways,B6,20409.0,N529JB,nan,nan,nan,nan,nan,nan,mid_route,JetBlue Airways,nan
999777,TRAIN_999777,9,27,nan,nan,0,0,STS,15023,California,SEA,14747,Washington,618.0,SkyWest Airlines Inc,AS,20304.0,N403SY,Delayed,nan,nan,nan,nan,nan,short_route,SkyWest Airlines Inc.,nan
999815,TRAIN_999815,12,1,nan,nan,0,0,SEA,14747,Washington,BOI,10713,Idaho,399.0,Horizon Air,AS,19687.0,N401QX,nan,nan,nan,nan,nan,nan,short_route,Delta Air Lines Inc.,nan
999911,TRAIN_999911,7,14,nan,nan,0,0,HOU,12191,Texas,DEN,11292,Colorado,883.0,Southwest Airlines Co,WN,19393.0,N935WN,nan,nan,nan,nan,nan,nan,mid_route,Southwest Airlines Co.,nan


In [120]:
train[['Estimated_Departure_Time',	'Estimated_Arrival_Time']].value_counts().index[0][0]

'nan'

# dummy

In [99]:
dum = train['Estimated_Arrival_Time_HH:MM'].iloc[1].split(':')
datetime.strptime(train['Estimated_Arrival_Time_HH:MM'].iloc[1], "%H:%M") - timedelta(minutes=1200) 

datetime.datetime(1899, 12, 31, 14, 24)

In [12]:
change_name_list = check[check['Airline'] == 'Southwest Airlines Co.']['NAME'].dropna().unique()
for cnl in change_name_list:
    check['NAME'].replace(cnl, 'Southwest Airlines Co.', inplace=True)
    
check[check['Airline'] == 'Southwest Airlines Co.']['NAME'].dropna().unique()

array(['Southwest Airlines Co.'], dtype=object)

In [61]:

import itertools
import pandas as pd

master_dil = 'MASTER2020.txt'
data = []
with open(master_dil, 'r', encoding = "utf-8-sig") as file:
           data.append(file.readlines())
           
data = list(itertools.chain(*data))  
data = [line.split(',') for line in data]
col = data[0]
#col_store = col.copy()
#data = [i[:35] for i in data]
#len(col_store)
df = pd.DataFrame(data, columns = col)
Airline2 = df[['N-NUMBER', 'NAME']].rename(columns={'N-NUMBER' : 'Tail_Number', 'NAME' : 'NAME2'})
df['Tail_Number'] = ['N' + i for i in df['N-NUMBER']]

Airline = df[['Tail_Number', 'NAME']]

In [62]:

train_ar = pd.merge(ddata, Airline, left_on = 'Tail_Number', right_on = 'Tail_Number', how = 'left')
train_ar = pd.merge(train_ar, Airline2, left_on = 'Tail_Number', right_on = 'Tail_Number', how = 'left')

NameError: name 'ddata' is not defined

In [53]:

train_ar = train_ar[['Airline', 'Tail_Number', 'Tail_Number','NAME', 'NAME2']]


In [54]:
train_ar['Airline'] = train_ar['Airline'].str.lower()
train_ar['Airline'] = train_ar['Airline'].str.split('.').str[0]
train_ar['NAME'] = train_ar['NAME'].str.lower()



In [55]:
train_ar = train_ar.fillna('None')
train_ar = train_ar.replace('None', None)

In [56]:
def apy(x):
    if x['Airline'] != None:
        x['NAME'] = x['Airline']
    return x
ar = train_ar.drop_duplicates().apply(apy, axis=1)

In [57]:
check = ar[ar['NAME'].isna()]

check[check['NAME2'].isna()]

,Airline,Tail_Number,Tail_Number,NAME,NAME2
6348,None,N158GJ,N158GJ,None,None
7451,None,N172GJ,N172GJ,None,None
14668,None,N3767,N3767,None,None
25023,None,N606CZ,N606CZ,None,None
34600,None,N369CA,N369CA,None,None
...,...,...,...,...,...
976233,None,N954DL,N954DL,None,None
978777,None,N9621A,N9621A,None,None
983324,None,406NV,406NV,None,None
996309,None,N501AE,N501AE,None,None


In [28]:
Airline[Airline['Tail_Number'] =='N3767']

,Tail_Number,NAME


In [109]:
(datetime.strptime(train['Estimated_Arrival_Time_HH:MM'].iloc[1], "%H:%M") + timedelta(minutes=2000) )

datetime.datetime(1900, 1, 2, 19, 44)

In [14]:
check = train[~((train['Estimated_Departure_Time'].isin(['nan'])) | (train['Estimated_Arrival_Time'].isin(['nan'])))]

In [20]:
check1 = train[(train['Estimated_Departure_Time'] == 'nan') & (train['Estimated_Arrival_Time'] != 'nan')]

In [21]:
check1.groupby(['Origin_Airport','Destination_Airport', 'Airline'])['Estimated_Arrival_Time'].value_counts()

Origin_Airport  Destination_Airport  Airline               Estimated_Arrival_Time
ABE             ATL                  Delta Air Lines Inc   811.0                     1
                                                           816.0                     1
                                                           820.0                     1
                                     Endeavor Air Inc      819.0                     1
                                                           833.0                     1
                                                                                    ..
YUM             PHX                  SkyWest Airlines Inc  1832.0                    1
                                                           1929.0                    1
                                                           2110.0                    1
                                                           2113.0                    1
                                                

# 전처리 변경

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',None)

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

In [3]:

import datetime
def to_time(time_list):
    Time = pd.Series(time_list).astype(str).str.zfill(4)
    Time = Time.replace('2400','0000')
    return [datetime.datetime.strptime(i, '%H%M').strftime("%H:%M") if i != '0nan' else np.NaN for i in Time] 

time_list = [i if str(i) == 'nan' else str(int(i))  for i in train['Estimated_Departure_Time'] ]
time_list1 = [i if str(i) == 'nan' else str(int(i))  for i in train['Estimated_Arrival_Time'] ]
train['Estimated_Departure_Time_HH:MM'] = to_time(time_list)
train['Estimated_Arrival_Time_HH:MM'] = to_time(time_list1)
train


,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Time_HH:MM,Estimated_Arrival_Time_HH:MM
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,NaN,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,07:40,10:24
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,16:10,18:05
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,09:05,17:35
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,09:00,10:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,09:36,12:43
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,09:20,10:28
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,08:00,13:40
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,16:13,18:24


In [5]:

# 예상 비행시간 만들기 (분으로 만들기)
train_est_time = []
for i,j in zip(train['Estimated_Arrival_Time_HH:MM'], train['Estimated_Departure_Time_HH:MM']):
    if str(i) != 'nan' and str(j) != 'nan':
        if (datetime.datetime.strptime(str(i), "%H:%M") - datetime.datetime.strptime(str(j), "%H:%M")).total_seconds()/60 >= 0:
            train_est_time.append((datetime.datetime.strptime(str(i), "%H:%M") - datetime.datetime.strptime(str(j), "%H:%M")).total_seconds()/60)
        else:
            time = datetime.datetime.strptime(str(i), "%H:%M") - datetime.datetime.strptime(str(j), "%H:%M") + datetime.datetime.strptime('23:59', "%H:%M") + datetime.timedelta(minutes=1)
            train_est_time.append(datetime.timedelta(hours=time.hour,minutes=time.minute ).total_seconds()/60)
    else:
        train_est_time.append(np.NaN)

In [6]:

train['Estimated_Time'] = train_est_time

In [8]:
def time_gb(x):

  if x >= 600 and x <= 659:
    return '0600-0659'
  elif x>=1400 and x<=1459:
    return '1400-1459'
  elif x>=1200 and x<=1259:
    return '1200-1259'
  elif x>=1500 and x<=1559:
    return '1500-1559'
  elif x>=1900 and x<=1959:
    return '1900-1959'
  elif x>=900 and x<=959:
    return '0900-0959'
  elif x>=1000 and x<=1059:
    return  '1000-1059'
  elif x>=2000 and x<=2059:
    return '2000-2059'
  elif x>=1300 and x<=1359:
    return '1300-1359'
  elif x>=1100 and x<=1159:
    return '1100-1159'
  elif x>=800 and x<=859:
    return '0800-0859'
  elif x>=2200 and x<=2259:
    return '2200-2259'
  elif x>=1600 and x<=1659:
    return '1600-1659'
  elif x>=1700 and x<=1759:
    return '1700-1759'
  elif x>=2100 and x<=2159:
    return '2100-2159'
  elif x>=700 and x<=759:
    return '0700-0759'
  elif x>=1800 and x<=1859:
    return '1800-1859'
  elif x>=1 and x<=559:
    return '0001-0559'
  elif x>=2300 and x<=2400:
    return '2300-2400'

# 예상 출발, 도착시간 그룹 화
train['Dep_time_gb'] = train['Estimated_Departure_Time'].dropna().apply(time_gb)
train['Arr_time_gb'] = train['Estimated_Arrival_Time'].dropna().apply(time_gb)
train

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Time_HH:MM,Estimated_Arrival_Time_HH:MM,Estimated_Time,Dep_time_gb,Arr_time_gb
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,07:40,10:24,164.0,0700-0759,1000-1059
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,16:10,18:05,115.0,1600-1659,1800-1859
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,09:05,17:35,510.0,0900-0959,1700-1759
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,09:00,10:19,79.0,0900-0959,1000-1059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,09:36,12:43,187.0,0900-0959,1200-1259
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,09:20,10:28,68.0,0900-0959,1000-1059
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,08:00,13:40,340.0,0800-0859,1300-1359
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,16:13,18:24,131.0,1600-1659,1800-1859


In [9]:

def distance_gb(x):
  if x < 700 :
    return 'short_route'
  elif x>=700 and x<3000:
    return 'mid_route'
  elif x>=3000:
    return 'long_route'    


# 예상 출발, 도착시간 그룹 화
train['route_gb'] = train['Distance'].apply(distance_gb)
train

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Time_HH:MM,Estimated_Arrival_Time_HH:MM,Estimated_Time,Dep_time_gb,Arr_time_gb,route_gb
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,NaN,NaN,NaN,NaN,NaN,short_route
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,07:40,10:24,164.0,0700-0759,1000-1059,mid_route
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,16:10,18:05,115.0,1600-1659,1800-1859,short_route
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,09:05,17:35,510.0,0900-0959,1700-1759,mid_route
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,09:00,10:19,79.0,0900-0959,1000-1059,short_route
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,09:36,12:43,187.0,0900-0959,1200-1259,short_route
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,09:20,10:28,68.0,0900-0959,1000-1059,short_route
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,08:00,13:40,340.0,0800-0859,1300-1359,mid_route
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,16:13,18:24,131.0,1600-1659,1800-1859,short_route


In [10]:
origin_state = train[['Origin_Airport','Origin_Airport_ID','Origin_State']].dropna()
origin_state = origin_state.drop_duplicates()
origin_state.columns = ['Origin_Airport','Origin_Airport_ID','F_Origin_State']
Destination_state = train[['Destination_Airport','Destination_Airport_ID','Destination_State']].dropna()
Destination_state = Destination_state.drop_duplicates()
Destination_state.columns = ['Destination_Airport','Destination_Airport_ID','F_Destination_State']


train = pd.merge(train, origin_state, on = ['Origin_Airport','Origin_Airport_ID'], how = 'outer')
train = pd.merge(train, Destination_state, on = ['Destination_Airport','Destination_Airport_ID'], how = 'outer')
train

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Time_HH:MM,Estimated_Arrival_Time_HH:MM,Estimated_Time,Dep_time_gb,Arr_time_gb,route_gb,F_Origin_State,F_Destination_State
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,NaN,NaN,NaN,NaN,NaN,short_route,Oklahoma,Texas
1,TRAIN_000827,7,16,550.0,715.0,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N733SA,NaN,05:50,07:15,85.0,0001-0559,0700-0759,short_route,Oklahoma,Texas
2,TRAIN_001452,1,13,1405.0,1535.0,0,0,OKC,13851,Oklahoma,HOU,12191,NaN,419.0,Southwest Airlines Co.,WN,19393.0,N204WN,NaN,14:05,15:35,90.0,1400-1459,1500-1559,short_route,Oklahoma,Texas
3,TRAIN_004015,5,17,1050.0,1215.0,0,0,OKC,13851,Oklahoma,HOU,12191,NaN,419.0,Southwest Airlines Co.,WN,19393.0,N246LV,NaN,10:50,12:15,85.0,1000-1059,1200-1259,short_route,Oklahoma,Texas
4,TRAIN_009325,8,14,600.0,725.0,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7706A,NaN,06:00,07:25,85.0,0600-0659,0700-0759,short_route,Oklahoma,Texas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_896461,11,26,1310.0,NaN,0,0,JMS,12519,North Dakota,DVL,11447,North Dakota,83.0,NaN,UA,20304.0,N915SW,NaN,13:10,NaN,NaN,1300-1359,NaN,short_route,North Dakota,North Dakota
999996,TRAIN_908847,1,5,1320.0,1408.0,0,0,JMS,12519,NaN,DVL,11447,North Dakota,83.0,SkyWest Airlines Inc.,UA,20304.0,N928EV,Not_Delayed,13:20,14:08,48.0,1300-1359,1400-1459,short_route,North Dakota,North Dakota
999997,TRAIN_911180,4,5,1315.0,1403.0,0,0,JMS,12519,North Dakota,DVL,11447,North Dakota,83.0,SkyWest Airlines Inc.,UA,20304.0,N951SW,NaN,13:15,14:03,48.0,1300-1359,1400-1459,short_route,North Dakota,North Dakota
999998,TRAIN_947096,11,11,1310.0,1355.0,0,0,JMS,12519,North Dakota,DVL,11447,North Dakota,83.0,SkyWest Airlines Inc.,UA,20304.0,N701BR,NaN,13:10,13:55,45.0,1300-1359,1300-1359,short_route,North Dakota,North Dakota


In [11]:

mask = (train['Destination_Airport'] == 'YNG')&(train['Destination_Airport_ID'] == 16133)
value = 'Indiana'

train.loc[mask,'F_Destination_State'] = train.loc[mask,'F_Destination_State'].fillna(value) 

In [63]:

import itertools

master_dil = 'MASTER2.txt'
df = []
with open(master_dil, 'r', encoding = "utf-8-sig") as file:
           df.append(file.readlines())
           
df = list(itertools.chain(*df))  
df = [line.split(',') for line in df]
col = df[0]
#col_store = col.copy()
#df = [i[:35] for i in df]
#len(col_store)
df = pd.DataFrame(df, columns = col)
Airline2 = df[['N-NUMBER', 'NAME']].rename(columns={'N-NUMBER' : 'Tail_Number', 'NAME' : 'NAME2'})

In [64]:
df['Tail_Number'] = ['N' + i for i in df['N-NUMBER']]

Airline = df[['Tail_Number', 'NAME']]


train_ar = pd.merge(train, Airline, left_on = 'Tail_Number', right_on = 'Tail_Number', how = 'left')
ar = pd.merge(train_ar, Airline2, left_on = 'Tail_Number', right_on = 'Tail_Number', how = 'left')

In [ ]:
for base in ar['Airline'].dropna().unique():
    change_name_list = ar[ar['Airline'] == base]['NAME'].dropna().unique()
    if len(change_name_list) != 0:
        for cnl in change_name_list:
            ar['NAME'].replace(cnl, base, inplace=True)
            
        
    change_name_list = ar[ar['Airline'] == base]['NAME2'].dropna().unique()
    if len(change_name_list) != 0:
        for cnl in change_name_list:
            ar['NAME2'].replace(cnl, base, inplace=True)

In [70]:
for base in ar['Airline'].dropna().unique():
    ar.loc[(ar['Airline'] == base) & ~(ar['NAME'].isin(['nan'])), 'NAME'] = base

In [8]:
check = ar[['Airline', 'Tail_Number', 'Tail_Number','NAME']]#[train['Tail_Number'].astype(str) == 'nan']

KeyError: "['NAME'] not in index"

In [4]:
dum = train.__deepcopy__()

In [76]:

dum = train.__deepcopy__()

master_dil = 'MASTER2.txt'
df = []
with open(master_dil, 'r', encoding = "utf-8-sig") as file:
    df.append(file.readlines())
    
df = list(itertools.chain(*df))  
df = [line.split(',') for line in df]
col = df[0]
df = pd.DataFrame(df, columns = col)
Airline2 = df[['N-NUMBER', 'NAME']].rename(columns={'N-NUMBER' : 'Tail_Number', 'NAME' : 'NAME2'})
df['Tail_Number'] = ['N' + i for i in df['N-NUMBER']]
Airline = df[['Tail_Number', 'NAME']]

dum = pd.merge(dum, Airline, left_on = 'Tail_Number', right_on = 'Tail_Number', how = 'left')
ar = pd.merge(dum, Airline2, left_on = 'Tail_Number', right_on = 'Tail_Number', how = 'left')

ar.loc[ar['Tail_Number'] == 'N297AK', 'NAME'] = 'ALASKA AIRLINES INC'
ar.loc[ar['Tail_Number'] == '276NV', 'NAME'] = 'ALLEGIANT AIR LLC'

ar = ar.fillna('nan')

# DB-NAME을 Train Airline 기준으로 바꿈
for base in tqdm(ar.loc[~ar['Airline'].isin(['nan']), 'Airline'].unique()):
    name1 = ar.loc[(ar['Airline'] == base) & ~(ar['NAME'].isin(['nan'])) & ~(ar['NAME'].isin(ar['Airline'].unique())), 'NAME'].unique()
    name2 = ar.loc[(ar['Airline'] == base) & ~(ar['NAME2'].isin(['nan'])) & ~(ar['NAME2'].isin(ar['Airline'].unique())), 'NAME2'].unique()
    ar['NAME'].replace(name1, base, inplace=True)
    ar['NAME2'].replace(name2, base, inplace=True)
    
    
def apy(x):
    if x['NAME'] != 'nan':
        x['Airline'] = x['NAME']
    elif (x['NAME'] == 'nan') & (x['NAME2'] != 'nan'):
        x['Airline'] = x['NAME2']
    return x

ar.loc[ar['Airline'] == 'nan'] = ar.loc[ar['Airline'] == 'nan'].apply(apy, axis=1)

for n in tqdm(ar[ar['Airline'] == 'nan']['Tail_Number'].unique()):
    if ar.loc[ar['Tail_Number'] == n, ['Airline', 'Carrier_ID(DOT)']].shape[0] != ar.loc[(ar['Tail_Number'] == n) & ~(ar['Airline'].isin(['nan'])), ['Airline', 'Carrier_ID(DOT)']].shape[0]:
        ar.loc[ar['Tail_Number'] == n, 'Airline'] = ar.loc[(ar['Tail_Number'] == n) & ~(ar['Airline'].isin(['nan'])), 'Airline'].unique()[0]
        ar.loc[ar['Tail_Number'] == n, 'Carrier_ID(DOT)'] = ar.loc[(ar['Tail_Number'] == n) & ~(ar['Airline'].isin(['nan'])), 'Carrier_ID(DOT)'].unique()[0]

# ar['Airline'] = ar['Airline'].str.rstrip()
# ar['Airline'] = ar['Airline'].str.split('.').str[0]

# # ID 다시채움
# for airline in tqdm(ar['Airline'].unique()):
#     id = ar.loc[(ar['Airline'] == airline) & ~(ar['Carrier_ID(DOT)'].isin(['nan'])), 'Carrier_ID(DOT)'].unique()
#     ar.loc[ar['Airline'] == airline, 'Carrier_ID(DOT)'] = id[0]

  0%|          | 0/28 [00:00<?, ?it/s]

[]


  4%|▎         | 1/28 [00:01<00:28,  1.05s/it]

[]


  7%|▋         | 2/28 [00:01<00:19,  1.31it/s]

...

[]


100%|██████████| 28/28 [00:11<00:00,  2.42it/s]

[]



 35%|███▌      | 46/131 [00:14<00:27,  3.15it/s]


KeyboardInterrupt: 

In [75]:
ar['Airline'].unique()

array(['Southwest Airlines Co.', 'SkyWest Airlines Inc.',
       'American Airlines Inc.', 'United Air Lines Inc.',
       'Republic Airlines', 'ExpressJet Airlines Inc.',
       'Delta Air Lines Inc.', 'Air Wisconsin Airlines Corp',
       'Horizon Air', 'JetBlue Airways', 'Spirit Air Lines',
       'Alaska Airlines Inc.', 'Mesa Airlines Inc.',
       'Frontier Airlines Inc.', 'Trans States Airlines',
       'Endeavor Air Inc.', 'Comair Inc.', 'Capital Cargo International',
       'Envoy Air', 'Hawaiian Airlines Inc.', 'Allegiant Air',
       'Virgin America', 'Peninsula Airways Inc.', 'Compass Airlines',
       'GoJet Airlines, LLC d/b/a United Express',
       'Commutair Aka Champlain Enterprises, Inc.',
       'Empire Airlines Inc.', 'Cape Air', 'nan'], dtype=object)

In [72]:
ar['NAME2'].unique()

array(['nan', 'Allegiant Air'], dtype=object)

In [13]:
ar.loc[ar['Airline'] == 'nan', 'Tail_Number'].unique().shape

(4880,)

In [34]:
def apy(x):
    if x['NAME'] != 'nan':
        x['Airline'] = x['NAME']
    elif (x['NAME'] == 'nan') & (x['NAME2'] != 'nan'):
        x['Airline'] = x['NAME2']
        
    return x
    
    # else:
    #     # print('?')
    #     pass
        
ar.loc[ar['Airline'] == 'nan'] = ar.loc[ar['Airline'] == 'nan'].apply(apy, axis=1)

In [37]:
ar[ar['Airline'] == 'nan']['Tail_Number'].to_csv('crwaling_list.csv', index=False)

In [71]:
count = 0
for n in ar[ar['Airline'] == 'nan']['Tail_Number'].unique():
    if ar.loc[ar['Tail_Number'] == n, ['Airline', 'Carrier_ID(DOT)']].shape[0] != ar.loc[(ar['Tail_Number'] == n) & ~(ar['Airline'].isin(['nan'])), ['Airline', 'Carrier_ID(DOT)']].shape[0]:
        count +=1
        
print(count)

131


In [57]:
print(ar.loc[ar['Tail_Number'] == n, ['Airline', 'Carrier_ID(DOT)']].shape[0])
print(ar.loc[(ar['Tail_Number'] == n) & ~(ar['Airline'].isin(['nan'])), ['Airline', 'Carrier_ID(DOT)']].shape[0])

54
53


In [52]:
ar.loc[ar['Tail_Number'] == n, ['Airline', 'Carrier_ID(DOT)']].dropna()

,Airline,Carrier_ID(DOT)
56352,Comair Inc.,20397.0
62611,Comair Inc.,20397.0
70541,Comair Inc.,20397.0
80345,Comair Inc.,20397.0
97346,Comair Inc.,20397.0
111126,Comair Inc.,20397.0
114712,Comair Inc.,20397.0
124916,Comair Inc.,20397.0
151825,Comair Inc.,20397.0
155775,Comair Inc.,20397.0


In [83]:
train[train['Estimated_Arrival_Time'] == 'nan']

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Time_HH:MM,Estimated_Arrival_Time_HH:MM,Estimated_Time,Dep_time_gb,Arr_time_gb,route_gb,NAME,NAME2
0,TRAIN_000000,4,15,nan,nan,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co,WN,19393.0,N7858A,nan,nan,nan,nan,nan,nan,short_route,Southwest Airlines Co.,nan
5,TRAIN_000005,4,13,1545.0,nan,0,0,EWR,11618,New Jersey,DCA,11278,Virginia,199.0,Republic Airlines,UA,20452.0,N657RW,Not_Delayed,15:45,nan,nan,1500-1559,nan,short_route,Republic Airlines,nan
20,TRAIN_000020,7,2,620.0,nan,0,0,BWI,10821,Maryland,BOS,10721,Massachusetts,369.0,Southwest Airlines Co,WN,19393.0,N292WN,nan,06:20,nan,nan,0600-0659,nan,short_route,Southwest Airlines Co.,nan
24,TRAIN_000024,11,28,1030.0,nan,0,0,TPA,15304,Florida,BHM,10599,Alabama,459.0,Southwest Airlines Co,WN,19393.0,N921WN,nan,10:30,nan,nan,1000-1059,nan,short_route,Southwest Airlines Co.,nan
28,TRAIN_000028,8,7,930.0,nan,0,0,SAN,14679,California,LAS,12889,Nevada,258.0,Spirit Air Lines,NK,20416.0,N628NK,Not_Delayed,09:30,nan,nan,0900-0959,nan,short_route,Southwest Airlines Co.,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999932,TRAIN_999932,1,15,2110.0,nan,0,0,DFW,11298,Texas,BNA,10693,Tennessee,631.0,American Airlines Inc,AA,19805.0,N809NN,nan,21:10,nan,nan,2100-2159,nan,short_route,Southwest Airlines Co.,nan
999946,TRAIN_999946,3,6,1925.0,nan,0,0,MDW,13232,Illinois,TPA,15304,Florida,997.0,Southwest Airlines Co,WN,19393.0,N8715Q,nan,19:25,nan,nan,1900-1959,nan,mid_route,Southwest Airlines Co.,nan
999965,TRAIN_999965,5,22,1555.0,nan,0,0,BOS,10721,Massachusetts,BNA,10693,Tennessee,942.0,Republic Airlines,DL,20452.0,N215JQ,nan,15:55,nan,nan,1500-1559,nan,mid_route,Republic Airlines,nan
999967,TRAIN_999967,12,30,nan,nan,0,0,BNA,10693,Tennessee,ORD,13930,Illinois,409.0,SkyWest Airlines Inc,UA,20304.0,N127SY,nan,nan,nan,nan,nan,nan,short_route,SkyWest Airlines Inc.,nan
